Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [17]:
%%sql
DROP TABLE if exists Positions;
CREATE TABLE Positions(id integer(0,0) NOT NULL PRIMARY KEY, Name varchar(0,0));
DROP TABLE if exists Teams;
CREATE TABLE Teams(id integer(0,0) NOT NULL PRIMARY KEY, Name varchar(0,0),
                   Location varchar(0,0), Coach_id varchar(0,0), FOREIGN KEY (Coach_id) REFERENCES Coaches(id));
DROP TABLE if exists Players;
CREATE TABLE Players(id integer(0,0) NOT NULL PRIMARY KEY, Name varchar(0,0), Age integer(0,0), Position_id varchar(0,0) NOT NULL,
                     Team_id varchar(0,0) NOT NULL, Start_date integer(0,0),
                     FOREIGN KEY (Position_id) REFERENCES Positions(id),
                     FOREIGN KEY (Team_id) REFERENCES Teams(id));
DROP TABLE if exists Coaches;
CREATE TABLE Coaches(id integer(0,0) PRIMARY KEY, name varchar(0,0));
DROP TABLE if exists PlayersArchive;
CREATE TABLE PlayersArchive(Player_id varchar(0,0), Team_id varchar(0,0), start_date integer(0,0), end_date integer(0,0),
                            FOREIGN KEY (Player_id) REFERENCES Players(id),
                            FOREIGN KEY (Team_id) REFERENCES Teams(id));

0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
Done.
0 rows affected.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [19]:
%%sql
INSERT INTO Positions VALUES (1, 'goalkeeper');
INSERT INTO Positions VALUES (2, 'defender');
INSERT INTO Positions VALUES (3, 'midfielder');
INSERT INTO Positions VALUES (4, 'forward');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [35]:
import sqlite3
db=sqlite3.connect('test1.db')
def check_coach(name):
    cur = db.cursor()
    cur.execute('''select id from Coaches where name = ?''', (name,))
    try:
        ID = cur.fetchone()[0]
        return ID
    except TypeError:
        print('Incorrect name')
        return -1
    
def insert_coach(Name):
    chk_chc = check_coach(Name)
    if chk_chc != -1:
        return -1
    cur = db.cursor()
    cur.execute( '''
    SELECT COALESCE(MAX(id)+1, 1) FROM Coaches''')
    ID = cur.fetchone()[0]
    cur.execute('''
        INSERT INTO Coaches(id, name) VALUES(?,?)''', (ID,Name))
    db.commit()


    

# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [37]:
def insert_team(name, location, coach_name):
    c_id = check_coach(coach_name)
    cur = db.cursor()
    cur.execute('''
    SELECT COALESCE(MAX(id) + 1, 1) FROM Teams''')
    ID = cur.fetchone()[0];
    cur.execute('''
        INSERT INTO Teams VALUES(?,?,?,?)''',(ID, name, location, c_id))
    db.commit()
insert_team('VP','Poland','Keku4')
insert_team('Nagibatory', 'Russia', 'John')
insert_team('Numbania', 'India', 'Zimbabwe')

# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [53]:
def check_team(name):
    cur = db.cursor()
    cur.execute('''select id from Teams where Name = ?''', (name,))
    try:
        ID = cur.fetchone()[0]
        return ID
    except TypeError:
        print('Wrong team')
        return -1
    
def update_teams(name, *args):
    team_id = check_team(name)
    cur = db.cursor()
    if len(args)>0:
        c_name = check_coach(args[0])
    if len(args) == 2:
        loc = args[1]
        cur.execute('''update Teams set Location = ?, Coach_id = ? where id = ?''', (loc, c_name, team_id))
    else:
        cur.execute('''update Teams set Coach_id = ? where id = ?''', (c_name, team_id))
    db.commit()
update_teams('VP','John')
update_teams('Nagibatory', 'Keku4', 'Poland')

# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [60]:
import datetime
def pos_check(name):
    cur = db.cursor()
    cur.execute('''select id from Positions where Name = ?''',(name,))
    try:
        ID = cur.fetchone()[0]
        return ID
    except TypeError:
        print('Wrong position')
        return -1

def insert_player(name, age, position, *args):
    pos_id = pos_check(position)
    if pos_id == -1:
        return -1
    team_id = check_team(args[0])
    if team_id == -1:
        return -1
    if age < 10 or age > 50:
        print('Wrong age')
        return -1
    if len(args) == 2:
        date = args[1]
    else:
        date = datetime.date.today()
    cur = db.cursor()
    cur.execute('''
    SELECT COALESCE(MAX(id) + 1, 1) FROM Players''')
    ID = cur.fetchone()[0];
    cur.execute('insert into Players(id, Name, Age, Position_id, Team_id, Start_date) values(?,?,?,?,?,?)',(ID, name, age, pos_id, team_id, date))
    db.commit()
insert_player('Sawa', 19, 'goalkeeper', 'VP')

# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [70]:
def player_transfer(name, team, date):
    cur = db.cursor()
    cur.execute('select id from Players where Name = ?', (name,))
    player_id = cur.fetchone()[0]
    cur.execute('select Team_id from Players where Name = ?', (name,))
    old_team_id = cur.fetchone()[0]
    cur.execute('select Start_date from Players where Name = ?', (name,))
    start = cur.fetchone()[0]
    cur.execute('insert into PlayersArchive values(?,?,?,?)',(player_id, old_team_id, start, date))
    team_id = check_team(team)
    cur.execute('update Players set Team_id = ?, Start_date = ? where id = ?',(team_id, date, player_id))
    db.commit()
    

In [71]:
player_transfer('Sawa', 'Nagibatory', '2017-05-09')